In [2]:
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split

In [4]:
# Define the path to the dataset
dataset_path = './testing'

# Create a list of file paths to the images
image_paths = [os.path.join(dataset_path, file) for file in os.listdir(dataset_path)]

In [5]:
image_paths

['./testing\\heatmap_0.png',
 './testing\\heatmap_1.png',
 './testing\\heatmap_2.png',
 './testing\\heatmap_3.png',
 './testing\\heatmap_4.png',
 './testing\\heatmap_5.png']

In [6]:
# Create a list of labels for each image
labels = [int(os.path.splitext(os.path.basename(path))[0].split('_')[-1]) for path in image_paths]

In [7]:
# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(500,500, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Preprocessing

In [9]:
# Create a function to load and preprocess an image
def load_and_preprocess_image(path, label):
    # Load the image
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    # Preprocess the image
    image = tf.image.resize(image, (500, 500))
    image /= 255.0
    return image, label

splitting

In [10]:
# Split the data into training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2,random_state=42)

In [11]:
train_paths, val_paths, train_labels, val_labels

(['./testing\\heatmap_5.png',
  './testing\\heatmap_2.png',
  './testing\\heatmap_4.png',
  './testing\\heatmap_3.png'],
 ['./testing\\heatmap_0.png', './testing\\heatmap_1.png'],
 [5, 2, 4, 3],
 [0, 1])

In [12]:
# Use the `tf.data` API to create datasets from the data
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))

In [13]:
# Map the `load_and_preprocess_image` function over each element in the dataset
train_dataset = train_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.map(load_and_preprocess_image)

# Batch the data
train_dataset = train_dataset.batch(32)
val_dataset = val_dataset.batch(32)

# New Code

import lib

In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from PIL import Image
import os

In [3]:
# The path to the folder containing the heatmaps
heatmap_folder = './resm'

In [4]:
# Get a list of all the heatmap filenames in the folder
heatmap_filenames = os.listdir(heatmap_folder)

In [6]:
# Load each heatmap as an image
heatmaps = []
for filename in heatmap_filenames:
    filepath = os.path.join(heatmap_folder, filename)
    heatmap = Image.open(filepath)
    heatmaps.append(np.array(heatmap))


labels

In [ ]:
# Extract the labels from the file names
labels = []
for name in heatmap_filenames:
    l = name.index('_') + 1
    r = name.index('.')
    class_id = int(name[l:r])
    labels.append(class_id)

In [8]:
# Convert the list of heatmaps into a numpy array
heatmaps = np.array(heatmaps)

Now normalize the data

In [9]:
# Normalize the heatmaps if necessary
heatmaps = heatmaps / 255.0

# Splitting

Splitting the data

In [10]:
# Split the data into training, validation, and test sets
train_data = heatmaps[:int(heatmaps.shape[0] * 0.7)]
val_data = heatmaps[int(heatmaps.shape[0] * 0.7):int(heatmaps.shape[0] * 0.9)]
test_data = heatmaps[int(heatmaps.shape[0] * 0.9):]

split the labels

In [ ]:
# Split the labels into training, validation, and test sets
train_labels = labels[:int(len(labels) * 0.7)]
val_labels = labels[int(len(labels) * 0.7):int(len(labels) * 0.9)]
test_labels = labels[int(len(labels) * 0.9):]

Define the architecture of CNN

In [11]:
# Define the architecture of the CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
          input_shape=heatmaps.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

compile the model

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

Train the model

In [ ]:
# Train the model
history = model.fit(train_data, epochs=10, batch_size=32,
                    validation_data=(val_data, val_labels))


Evaluate the model

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)